# Multiclass Random Forest Blosc

## Objetivos
* **[Dead End]** Crear un algoritmo de arboles de decisión basado en bosques aleatorios utilizando scikit-learn.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

%load_ext autoreload
%autoreload 2

%load_ext version_information
%version_information numpy, scipy, matplotlib, pandas

Software versions
Python 3.5.2 64bit [MSC v.1900 64 bit (AMD64)]
IPython 5.1.0
OS Windows 10 10.0.14393 SP0
numpy 1.11.3
scipy 0.19.0
matplotlib 2.0.0
pandas 0.19.2
Wed Mar 29 08:29:08 2017 Hora de verano romance

In [2]:
import os
import sys
sys.path.append("../src/")

from IPython.display import display
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()

pd.options.display.float_format = '{:,.3f}'.format
matplotlib.rcParams.update({'font.size': 12})

## Importando los datos de entramiento
Para ver como se crearon los datos de entrenamiento ir a [Training data generator](../deliver/training_data_generator.ipynb)

In [3]:
df = pd.read_csv('../data/training_data.csv', sep='\t')

In [4]:
df.head()

,BLZ_CRate,BLZ_CSpeed,BLZ_DSpeed,Block_Size,CL,CRate,CSpeed,Chunk_Number,Chunk_Size,Codec,...,LZ4_DSpeed,Max,Mean,Median,Min,Q1,Q3,Sd,Skew,Table
0,1.110,3.596,8.533,16.000,3.000,1.141,3.598,1.000,16.000,blosclz,...,8.606,27.146,5.762,3.093,-14.914,-0.990,12.773,8.290,0.555,0.000
1,1.110,3.596,8.533,32.000,3.000,1.239,6.426,1.000,16.000,lz4,...,8.606,27.146,5.762,3.093,-14.914,-0.990,12.773,8.290,0.555,0.000
2,1.110,3.596,8.533,0.000,2.000,1.232,6.367,1.000,16.000,lz4,...,8.606,27.146,5.762,3.093,-14.914,-0.990,12.773,8.290,0.555,0.000
3,1.110,3.596,8.533,"2,048.000",9.000,1.366,0.014,1.000,16.000,zstd,...,8.606,27.146,5.762,3.093,-14.914,-0.990,12.773,8.290,0.555,0.000
4,1.110,3.596,8.533,0.000,7.000,1.309,0.154,1.000,16.000,lz4hc,...,8.606,27.146,5.762,3.093,-14.914,-0.990,12.773,8.290,0.555,0.000


In [5]:
# INPUT OPTIONS TUNING
from sklearn.preprocessing import binarize 
df = df.assign(is_Table=binarize(df['Table'].values.reshape(-1,1), 0), 
               is_Columnar=binarize(df['Table'].values.reshape(-1,1), 1),
               is_Int=df['DType'].str.contains('int').astype(int),
               is_Float=df['DType'].str.contains('float').astype(int),
               is_String=(df['DType'].str.contains('S') | df['DType'].str.contains('U')).astype(int))


In [6]:
def aux_func(n):
    if n == 32 or n == 64:
        return n // 8
    else:
        return n
df['Type_Size'] = [aux_func(int(s)) for s in df['DType'].str[-2:]]

In [7]:
# OUTPUT OPTIONS TUNING
df = df.assign(Blosclz=(df['Codec'] == 'blosclz').astype(int),
               Lz4=(df['Codec'] == 'lz4').astype(int),
               Lz4hc=(df['Codec'] == 'lz4hc').astype(int),
               Snappy=(df['Codec'] == 'snappy').astype(int),
               Zstd=(df['Codec'] == 'zstd').astype(int),
               Shuffle=(df['Filter'] == 'shuffle').astype(int),
               Bitshuffle=(df['Filter'] == 'bitshuffle').astype(int))
               #CL1=df['CL'] == 1,
               #CL2=df['CL'] == 2,
               #CL3=df['CL'] == 3,
               #CL4=df['CL'] == 4,
               #CL5=df['CL'] == 5,
               #CL6=df['CL'] == 6,
               #CL7=df['CL'] == 7,
               #CL8=df['CL'] == 8,
               #CL9=df['CL'] == 9,
               #BS0=df['Block_Size'] == 0,
               #BS16=df['Block_Size'] == 16,
               #BS32=df['Block_Size'] == 32,
               #BS64=df['Block_Size'] == 64,
               #BS128=df['Block_Size'] == 128,
               #BS256=df['Block_Size'] == 256,
               #BS512=df['Block_Size'] == 512,
               #BS1024=df['Block_Size'] == 1024,
               #BS2048=df['Block_Size'] == 2048
#INTS = ['IN_CR', 'IN_CS', 'IN_DS', 'CL', 'Block_Size']
#df[INTS] = df[INTS].apply(pd.to_numeric, args=('ignore', 'integer'))

In [8]:
from sklearn.preprocessing import OneHotEncoder
# OUTPUT OPTIONS TUNING
enc_cl = OneHotEncoder()
enc_cl.fit(df['CL'].values.reshape(-1, 1))
new_cls = enc_cl.transform(df['CL'].values.reshape(-1, 1)).toarray()
enc_block = OneHotEncoder()
enc_block.fit(df['Block_Size'].values.reshape(-1, 1))
new_blocks = enc_block.transform(df['Block_Size'].values.reshape(-1, 1)).toarray()

for i in range(9):
    cl_label = 'CL' + str(i)
    block_label = 'Block' + str(i)
    df[cl_label] = new_cls[:, i]
    df[block_label] = new_blocks[:, i]
df['Block9'] = new_blocks[:, 9]

In [9]:
# SINGLE OUTPUT OPTION TUNING
df['Classes'] = df['Codec'] + '-' + df['Filter'] + '-' + df['CL'].apply(str)+ '-' + df['Block_Size'].apply(str)
class_map = df.drop_duplicates(subset=['Classes'])['Classes'].to_dict()
class_map = {el:i for i, el in enumerate(class_map.values()) }
df['Classes_ID'] = df['Classes'].apply(class_map.get)

In [10]:
#CHUNK_FEATURES = ["Table", "DType", "Chunk_Size", "Mean", "Median", "Sd", "Skew", "Kurt", "Min", "Max", "Q1", "Q3", "N_Streaks"]
IN_OPTIONS = ['IN_CR', 'IN_CS', 'IN_DS', 'is_Table', 'is_Columnar', 'is_Int', 'is_Float', 'is_String', 'Type_Size', 'Chunk_Size',
              'Mean', 'Median', 'Sd', 'Skew', 'Kurt', 'Min', 'Max', 'Q1', 'Q3', 'BLZ_CRate', 'BLZ_CSpeed', 'BLZ_DSpeed', 'LZ4_CRate',
              'LZ4_CSpeed', 'LZ4_DSpeed']
OUT_OPTIONS = ['Block0', 'Block1', 'Block2', 'Block3', 'Block4', 'Block5', 'Block6', 'Block7', 'Block8', 'Block9',
               'Blosclz', 'Lz4', 'Lz4hc', 'Snappy', 'Zstd', 'Shuffle', 'Bitshuffle',
               'CL0', 'CL1', 'CL2', 'CL3', 'CL4', 'CL5', 'CL6', 'CL7', 'CL8']
#OUT_OPTIONS = ['Block_Size', 'Blosclz', 'Lz4', 'Lz4hc', 'Snappy', 'Zstd',
#               'Shuffle', 'Bitshuffle', 'CL']

In [11]:
X, Y = df[IN_OPTIONS].values, df[['Classes_ID']].values.ravel()

In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
# [DIFFER] thresholds randomness instead of most discriminative
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y)
clf = RandomForestClassifier(n_estimators=100)
clf.fit(Xtrain, Ytrain)
Ypred = clf.predict(Xtest)

In [13]:
from sklearn import metrics
print(metrics.classification_report(Ytest, Ypred, digits=3))

             precision    recall  f1-score   support

          0      0.000     0.000     0.000         3
          2      0.000     0.000     0.000         3
          3      0.954     0.954     0.954        65
          4      0.000     0.000     0.000         3
          5      0.000     0.000     0.000         3
          6      0.000     0.000     0.000         2
          7      0.000     0.000     0.000         1
          8      0.250     0.200     0.222         5
          9      0.091     0.333     0.143         3
         11      0.250     0.333     0.286         3
         12      0.000     0.000     0.000         2
         13      0.000     0.000     0.000         3
         14      0.625     0.500     0.556        10
         15      0.000     0.000     0.000         5
         16      0.644     0.829     0.725        35
         17      0.000     0.000     0.000         1
         18      0.429     0.750     0.545         4
         19      0.000     0.000     0.000   

C:\Users\Shurberto\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Shurberto\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Insuficientes datos para plantear el problema como multiclass, además los datos de entrenamiento configurados de esta forma no están nada balanceados por lo que aunque tengamos suficientes datos seguiría siendo complicado obtener buenos resultados por este camino.